## Lesson 2: Connecting with a CRM

## Preparation 
<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh
%run ./ro_shared_data/lesson_2_prep.py lesson2

import os

agentId = os.environ['BEDROCK_AGENT_ID']
agentAliasId = os.environ['BEDROCK_AGENT_ALIAS_ID']
region_name = 'us-west-2'
lambda_function_arn = os.environ['LAMBDA_FUNCTION_ARN']

Resetting environment (if nessesary)
Found: mugs-customer-support-agent
Deleting alias: MyAgentAlias (ID: CCZTY0GASO)
Deletion initiated for alias: MyAgentAlias
Alias MyAgentAlias has been successfully deleted.
Deleting alias: AgentTestAlias (ID: TSTALIASID)
Deletion initiated for alias: AgentTestAlias
Alias AgentTestAlias has been successfully deleted.
Deleting agent: mugs-customer-support-agent (ID: VRLBJVA7GN)
Deletion initiated for agent: mugs-customer-support-agent
Waiting for agent mugs-customer-support-agent to be deleted...
Waiting for agent mugs-customer-support-agent to be deleted...
Agent mugs-customer-support-agent has been successfully deleted.
Agent reset process completed.
Deleting Lambda function: dlai-support-agent-I061F
Lambda function dlai-support-agent-I061F deleted successfully.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.
Lesson 2 Prep
Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent statu

## Start of lesson

In [2]:
import boto3
import uuid
from helper import *

In [3]:
sessionId = str(uuid.uuid4())
message = "My name is Mike, my mug is broken and I want a refund."

In [4]:
invoke_agent_and_print(
    agentId=agentId, 
    agentAliasId=agentAliasId, 
    inputText=message, 
    sessionId=sessionId
)

User: My name is Mike, my mug is broken and I want a refund.

Agent: Mike, I'm sorry to hear your mug is broken. To get a refund, I'll need
       to transfer you to one of our human customer support agents who
       can look into your order details and process the refund for
       you. Please hold while I connect you. Thank you for your
       patience.

Session ID: b7cc160c-cd59-48e7-9338-beb844e73451


<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; The file that is examined in the video is at  <code>./ro_shared_data/functions/lambda_stage_1.py</code> </p>

In [5]:
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)

In [6]:
create_agent_action_group_response = bedrock_agent.create_agent_action_group(
    actionGroupName='customer-support-actions',
    agentId=agentId,
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    functionSchema={
        'functions': [
            {
                'name': 'customerId',
                'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private information and must not be given to the user.',
                'parameters': {
                    'email': {
                        'description': 'Email address',
                        'required': False,
                        'type': 'string'
                    },
                    'name': {
                        'description': 'Customer name',
                        'required': False,
                        'type': 'string'
                    },
                    'phone': {
                        'description': 'Phone number',
                        'required': False,
                        'type': 'string'
                    },
                }
            },            
            {
                'name': 'sendToSupport',
                'description': 'Send a message to the support team, used for service escalation. ',
                'parameters': {
                    'custId': {
                        'description': 'customer ID',
                        'required': True,
                        'type': 'string'
                    },
                    'supportSummary': {
                        'description': 'Summary of the support request',
                        'required': True,
                        'type': 'string'
                    }
                }
            }
        ]
    },
    agentVersion='DRAFT',
)

In [7]:
create_agent_action_group_response

{'ResponseMetadata': {'RequestId': '5a974dfe-4781-4ef0-b461-3eb9478658fe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 28 Mar 2025 00:15:11 GMT',
   'content-type': 'application/json',
   'content-length': '1187',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5a974dfe-4781-4ef0-b461-3eb9478658fe',
   'x-amz-apigw-id': 'IHGCgGVYPHcEtBw=',
   'x-amzn-trace-id': 'Root=1-67e5ea0f-7a657eb5776d1f7b3949f3f7'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:637423536516:function:dlai-support-agent-8UDM0'},
  'actionGroupId': 'XZ6X0JL22O',
  'actionGroupName': 'customer-support-actions',
  'actionGroupState': 'ENABLED',
  'agentId': '7MHUJ8DSVB',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2025, 3, 28, 0, 15, 11, 736849, tzinfo=tzlocal()),
  'functionSchema': {'functions': [{'description': 'Get a customer ID given available details. At least one parameter must be sent to the function. This is private

In [8]:
actionGroupId = create_agent_action_group_response['agentActionGroup']['actionGroupId']


In [9]:
wait_for_action_group_status(
    agentId=agentId, 
    actionGroupId=actionGroupId,
    targetStatus='ENABLED'
)

Action Group status: ENABLED


'ENABLED'

In [10]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

wait_for_agent_status(
    agentId=agentId,
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
bedrock_agent.update_agent_alias(
    agentId=agentId,
    agentAliasId=agentAliasId,
    agentAliasName='MyAgentAlias',
)

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: UPDATING
Agent alias status: UPDATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


### Now use the agent with functions

In [12]:
sessionId = str(uuid.uuid4())
message = "My name is Mike (mike@mike.com), my mug is broken and I want a refund."

In [13]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=False
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent:  Hmm, it looks like I'm missing some details to properly route this to
       the support team. Let me ask the customer for some additional
       information.  Could you please provide more details about the
       broken mug, such as: - When did you purchase the mug? - Where
       did you purchase it from (online, in-store, etc.)? - Do you
       have any order or purchase details that could help us locate
       the transaction? - Any other relevant details about the issue
       with the mug?  With this additional information, I can better
       summarize the issue and route it to the appropriate support
       team to assist you with the refund request.

Session ID: ee78ee11-044c-4d44-8673-5b5069235f84


In [14]:
invoke_agent_and_print(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    enableTrace=True
)

User: My name is Mike (mike@mike.com), my mug is broken and I want a refund.

Agent: 
Agent's thought process:
  Okay, let me try processing this again with the information
  provided.  I was able to look up the customer's account details
  using the name and email, and the customer ID is 1006.  Since the
  customer is reporting a broken mug and requesting a refund, I will
  need to route this to the support team to properly handle the refund
  request and product issue.

Invocation Input:
  Type: ACTION_GROUP
  Action Group: customer-support-actions
  Function: sendToSupport
  Parameters: [{'name': 'supportSummary', 'type': 'string', 'value': 'Customer Mike (mike@mike.com, customer ID 1006) is reporting a broken mug and requesting a refund.'}, {'name': 'custId', 'type': 'string', 'value': '1006'}]

Observation:
  Type: ACTION_GROUP
  Action Group Output: {'error':'Details missing.'}

Agent's thought process:
  Hmm, it seems like I'm still missing some key details to properly
  route t